Vysvetlivky k stlpcom z: https://www.kaggle.com/fedesoriano/stroke-prediction-dataset?fbclid=IwAR1JNHGZbEhtzY0LdMW5YjkyT4JhRp1A8qzUhEFxai2b-2nRhw4JFQcNx-0

* id: unique identifier
* gender: "Male", "Female" or "Other"
* age: age of the patient
* hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
* heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease
* ever_married: "No" or "Yes"
* work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
* Residence_type: "Rural" or "Urban"
* avg_glucose_level: average glucose level in blood
* bmi: body mass index
* smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*
* stroke: 1 if the patient had a stroke or 0 if not
*Note: "Unknown" in smoking_status means that the information is unavailable for this patient

Nacitaj data:

In [89]:
df = read.csv("./data/healthcare-dataset-stroke-data.xls")
head(df)

id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
9046,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61,0,0,Yes,Self-employed,Rural,202.21,N/A,never smoked,1
31112,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79,1,0,Yes,Self-employed,Rural,174.12,24,never smoked,1
56669,Male,81,0,0,Yes,Private,Urban,186.21,29,formerly smoked,1


### Zakladne info o datach

Najskor nas zaujima vyskyt abnormalit ako chybajuce hodnoty alebo hodnoty ktore nie su realne. 

Z nizsie uvedeneho sumaru vidime napriklad, ze znak 'gender' ma 3 rozne kategorie, pricom sa ocakavaju len 2 a ze tretiu kategoriu ma len jedno pozorovanie. 

Dalej znak 'bmi' je uvedeny ako kategoricke data, hoci by sme ocakavali skor ciselne vyjadrenie.

In [90]:
str(df)

'data.frame':	5110 obs. of  12 variables:
 $ id               : int  9046 51676 31112 60182 1665 56669 53882 10434 27419 60491 ...
 $ gender           : Factor w/ 3 levels "Female","Male",..: 2 1 2 1 1 2 2 1 1 1 ...
 $ age              : num  67 61 80 49 79 81 74 69 59 78 ...
 $ hypertension     : int  0 0 0 0 1 0 1 0 0 0 ...
 $ heart_disease    : int  1 0 1 0 0 0 1 0 0 0 ...
 $ ever_married     : Factor w/ 2 levels "No","Yes": 2 2 2 2 2 2 2 1 2 2 ...
 $ work_type        : Factor w/ 5 levels "children","Govt_job",..: 4 5 4 4 5 4 4 4 4 4 ...
 $ Residence_type   : Factor w/ 2 levels "Rural","Urban": 2 1 1 2 1 2 1 2 1 2 ...
 $ avg_glucose_level: num  229 202 106 171 174 ...
 $ bmi              : Factor w/ 419 levels "10.3","11.3",..: 240 419 199 218 114 164 148 102 419 116 ...
 $ smoking_status   : Factor w/ 4 levels "formerly smoked",..: 1 2 2 3 2 1 2 2 4 4 ...
 $ stroke           : int  1 1 1 1 1 1 1 1 1 1 ...


In [91]:
summary(df)

       id           gender          age         hypertension    
 Min.   :   67   Female:2994   Min.   : 0.08   Min.   :0.00000  
 1st Qu.:17741   Male  :2115   1st Qu.:25.00   1st Qu.:0.00000  
 Median :36932   Other :   1   Median :45.00   Median :0.00000  
 Mean   :36518                 Mean   :43.23   Mean   :0.09746  
 3rd Qu.:54682                 3rd Qu.:61.00   3rd Qu.:0.00000  
 Max.   :72940                 Max.   :82.00   Max.   :1.00000  
                                                                
 heart_disease     ever_married         work_type    Residence_type
 Min.   :0.00000   No :1757     children     : 687   Rural:2514    
 1st Qu.:0.00000   Yes:3353     Govt_job     : 657   Urban:2596    
 Median :0.00000                Never_worked :  22                 
 Mean   :0.05401                Private      :2925                 
 3rd Qu.:0.00000                Self-employed: 819                 
 Max.   :1.00000                                                   
    

Identifikator nie je sam o sebe zaujimavy, ale ocakavame ze vsetky hodnoty tohto stlpca budu unikatne:

In [92]:
nrow(unique(df[c("id")])) == nrow(df[c("id")])

[1] TRUE

Stlpec 'gender' ma 3 unikatne hodnoty. Tretia hodnota je 'Other' a vyskytuje sa v jedinom pozorovani. Kedze sa jedna o jedine pozorovanie a nevieme pohlavie urcit, toto pozorovanie odstranime a budeme uvazovat len 2 pohlavia. 

In [93]:
unique(df[c("gender")])

,gender
1,Male
2,Female
3117,Other


In [95]:
df = df[df$gender != 'Other', ]

In [97]:
unique(df[c("gender")])

gender
Male
Female


Znak 'bmi' sa rozoznal ako kategoricky pre nerozoznane NA hodnoty. Preto ich transformujeme. Taktiez kategoriu 'Unknown' v znaku 'smoking_status' mozeme previest na NA pre konzistenciu.

In [98]:
df[] <- lapply(df, function(x) {
    is.na(levels(x)) <- levels(x) == "N/A"
    x
})

df[] <- lapply(df, function(x) {
    is.na(levels(x)) <- levels(x) == "Unknown"
    x
})

In [99]:
df['bmi'] = sapply(df['bmi'], as.character)
df['bmi'] = sapply(df['bmi'], as.numeric)

Teraz sa vieme pozriet, kde mame NA:

In [100]:
apply(df, 2, function(x) any(is.na(x)))

id            gender               age      hypertension 
            FALSE             FALSE             FALSE             FALSE 
    heart_disease      ever_married         work_type    Residence_type 
            FALSE             FALSE             FALSE             FALSE 
avg_glucose_level               bmi    smoking_status            stroke 
            FALSE              TRUE              TRUE             FALSE

In [101]:
summary(df)

       id           gender          age         hypertension    
 Min.   :   67   Female:2994   Min.   : 0.08   Min.   :0.00000  
 1st Qu.:17740   Male  :2115   1st Qu.:25.00   1st Qu.:0.00000  
 Median :36922   Other :   0   Median :45.00   Median :0.00000  
 Mean   :36514                 Mean   :43.23   Mean   :0.09748  
 3rd Qu.:54643                 3rd Qu.:61.00   3rd Qu.:0.00000  
 Max.   :72940                 Max.   :82.00   Max.   :1.00000  
                                                                
 heart_disease     ever_married         work_type    Residence_type
 Min.   :0.00000   No :1756     children     : 687   Rural:2513    
 1st Qu.:0.00000   Yes:3353     Govt_job     : 657   Urban:2596    
 Median :0.00000                Never_worked :  22                 
 Mean   :0.05402                Private      :2924                 
 3rd Qu.:0.00000                Self-employed: 819                 
 Max.   :1.00000                                                   
    